##SETUP

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!rm -r /content/transliteration
!git clone https://github.com/adkta/transliteration.git

rm: cannot remove '/content/transliteration': No such file or directory
Cloning into 'transliteration'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 137 (delta 74), reused 93 (delta 37), pack-reused 0 (from 0)
Receiving objects: 100% (137/137), 34.34 KiB | 6.87 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [3]:
!pip install g2p_en

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 100.2 MB/s eta 0:00:00
  Created wheel for distance: filename=Distance-0.1.3-py3-none-any.whl size=16256 sha256=0cadefe70a6782e5d90dd5b59bd210162a7d0604676d2a395f7a2e55cf9e8a94
  Stored in directory: /root/.cache/pip/wheels/24/a8/58/407063d8e5c1d4dd6594c99d12baa0108570b56a92325587dd
Successfully built distance


In [4]:
!pip install nepali-num2word

In [5]:
!git clone https://github.com/adkta/nepali_arabic_num_to_word.git

Cloning into 'nepali_arabic_num_to_word'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15 (delta 3), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (15/15), 4.65 KiB | 4.65 MiB/s, done.
Resolving deltas: 100% (3/3), done.


##ADD TRANSLITERATION DICTIONARY
Only for new transcripts

In [6]:
!ls /content/drive/MyDrive/MSICE/Predetermined_CS_Texts/

'Predetermined Texts.gdoc'   To_Send_Predetermined_Code-mixed_Texts_native.txt


In [10]:
from transliteration.transliterator import TranslitDict
from transliteration.transliterators import RomanToDevaTransliterator

current_translit_dict = TranslitDict.load(src_path = '/content/drive/MyDrive/MSICE/English_Nepali_CS_Data_Manual/Transliteration Dictionary/Roman_Devanagari_Translit_Dict.json')
translitr = RomanToDevaTransliterator(translit_dict = current_translit_dict)
add_translit_dict = TranslitDict.create(transcr_src = '/content/drive/MyDrive/MSICE/Predetermined_CS_Texts/To_Send_Predetermined_Code-mixed_Texts_native.txt', translitr = translitr)

INFO:transliteration:Extracting dictionary from file: /content/drive/MyDrive/MSICE/Predetermined_CS_Texts/To_Send_Predetermined_Code-mixed_Texts_native.txt...
Extracting dictionary from file: /content/drive/MyDrive/MSICE/Predetermined_CS_Texts/To_Send_Predetermined_Code-mixed_Texts_native.txt: 79it [01:10,  1.12it/s]


In [14]:
print(f"Current Dict Length: {len(current_translit_dict)}\nAdd Dict Length: {len(add_translit_dict)}")

Current Dict Length: 1354
Add Dict Length: 130


In [15]:
current_translit_dict.update(add_translit_dict)

In [16]:
print(f"New Dict Length: {len(current_translit_dict)}")

New Dict Length: 1427


In [17]:
current_translit_dict.export_to_json('/content/Roman_Devanagari_Translit_Dict.json')

In [18]:
!cp /content/Roman_Devanagari_Translit_Dict.json /content/drive/MyDrive/MSICE/English_Nepali_CS_Data_Manual/Transliteration\ Dictionary/

##GENERAL TRANSFORM LOGIC

In [19]:
from typing import Callable

import re
from transliteration.transliterator import TranslitDict


def transform_transcript(input_path: str, output_path: str, transform_fn: Callable[[str], str]) -> None:
    tknzr_pattern = re.compile(TranslitDict.PUNCT_SPACE_REGEX)
    with open(input_path, mode = 'r') as transcr_in, \
        open(output_path, mode = 'w') as transcr_out:
        for line in transcr_in:
            line = line.strip()
            audio_nm, label = line.split('\t')
            words = tknzr_pattern.split(label)
            numless_native_label = []
            for word in words:
                if not word:
                    continue
                numless_native_label.append(transform_fn(word))
            print(f"{audio_nm}\t{' '.join(numless_native_label)}", file=transcr_out)

##ORIGINAL NUMLESS TRANSCRIPT

1. Input original native transcript: /content/transcript.txt
2. Output is unpunctuated numberless native transcript. /content/native_numless_punctless_transcript.txt

i.e. All numerals are written in words

In [ ]:
!cp '/content/drive/MyDrive/MSICE/native_augmented_transcript.txt' /content/

In [21]:
from nepali_arabic_num_to_word.nepali_arabic_num_to_word import numeral_type, get_word_from_numeral

def num_2_word(word: str) -> str:
    num_type = numeral_type(word)
    if numeral_type:
        word = get_word_from_numeral(word, num_type)
    return word

In [23]:
# transform_transcript(input_path = "/content/native_augmented_transcript.txt", output_path = "/content/native_augmented_numless_punctless_transcript.txt", transform_fn = num_2_word)
transform_transcript(input_path = "/content/transcript-2.txt", output_path = "/content/native_augmented_numless_punctless_transcript.txt", transform_fn = num_2_word)

In [25]:
!head -20 native_augmented_numless_punctless_transcript.txt

PT_Ashish_Devkota_audio_1	सरकारी school हरुमा पढ्नेहरु back नलागिकनै continue regular college level मा pass गर्दै गईराको छ
PT_Ashish_Devkota_audio_2	private को students हरुले pass गर्न सकेन भन्न खोजेको होइन
PT_Ashish_Devkota_audio_3	young generations हरुको लागि English एकदम important छ
PT_Ashish_Devkota_audio_4	Even नेपाली subject पढाउने teachers हरु पनि motivation create गर्न बिचमा अंग्रेजी शब्दहरु प्रयोग गर्नुहुन्छ
PT_Ashish_Devkota_audio_5	दिपिका जी तपाई inspiration प्रेरणाको source हुनुहुन्छ
PT_Ashish_Devkota_audio_6	सरकार even prime minister पनि विकासको लागि budget थुप्रै छ भन्छ
PT_Ashish_Devkota_audio_7	तर NIST को plus two chain बिस्तारै अलि खस्किँदै जस्तो लागेको छ मलाई त
PT_Ashish_Devkota_audio_8	MG Sir त बिन्दास अहिले त total transformation हुनुभाछ नि पहिले त अलि strict हुनुहुन्थ्यो
PT_Ashish_Devkota_audio_9	Dubai को लागि भनेर Bahrain को लागि त vacancy नै छैन
PT_Ashish_Devkota_audio_10	Clients हरुले पुरै website visit नगरेसम्म हाम्रो कामको बारे थाहा हुँदैन
PT_Ashish_Devkota_aud

##ADD REDUCTION DICTIONARY (COMBINES TRANSLITERATION AND REDUCTION)

TASK 1: Create a reduction dictionary
The input to this task is numberless native transcript file
Output is a reduction dictionary

TASK 2: Create a reduced transcript file
The input is numberless native transcript file and reduction dictionary (if possible so the mapping can be reused)
Output is reduced transcript file

In [ ]:
# !cp '/content/drive/MyDrive/MSICE/English_Nepali_CS_Data_Manual/Transliteration Dictionary/Roman_Devanagari_Translit_Dict.json' /content/

In [ ]:
!head -4 /content/native_numless_punctless_transcript.txt > /content/test_transcript.txt

head: cannot open '/content/native_numless_punctless_transcript.txt' for reading: No such file or directory


In [ ]:
!head -4 /content/test_transcript.txt

In [38]:
from transliteration.transliterator import TranslitDict
from transliteration.reducers import get_reduced_devanagari_word
from transliteration.transliterators import RomanToDevaTransliterator
from tqdm import tqdm
from typing import Optional

import re

def create_reduc_dict(input_path: str, output_path: str, translit_dict_path: Optional[str] = None) -> None:
    tknzr_pattern = re.compile(TranslitDict.PUNCT_SPACE_REGEX)
    if translit_dict_path:
        translitr = RomanToDevaTransliterator(translit_dict = translit_dict_path)
    else:
        translitr = RomanToDevaTransliterator()

    with open(input_path, mode = 'r') as transcr_in, \
        open(output_path, mode = 'w') as transcr_out:
        for line in tqdm(transcr_in):
            line = line.strip()
            audio_nm, label = line.split('\t')
            words = tknzr_pattern.split(label)
            numless_native_label = []
            for word in words:
                if not word:
                    continue

                orig_word = word
                if translitr.for_transliteration(word):
                    word = translitr.translit(word)
                reduced_word = get_reduced_devanagari_word(word)

                if not reduced_word or reduced_word == orig_word:
                    continue

                print(f"{orig_word}\t{reduced_word}", file=transcr_out)

In [39]:
# Without transliteration dictionary file (A little slow)
# create_reduc_dict(input_path = "/content/native_augmented_numless_punctless_transcript.txt", output_path = "/content/test_reduc_dict.dat")
# With transiteration dictionary file (Faster)
# create_reduc_dict(input_path = "/content/native_augmented_numless_punctless_transcript.txt", output_path = "/content/test_reduc_dict.dat", translit_dict_path = "/content/Roman_Devanagari_Translit_Dict.json")

40it [00:00, 5969.26it/s]


In [29]:
!cp /content/drive/MyDrive/MSICE/English_Nepali_CS_Data_Manual/Transliteration\ Dictionary/Nep_Eng_Code-Mixed_Reduct_Dict.json /content/

In [30]:
current_reduc_dict = TranslitDict.load(src_path = '/content/Nep_Eng_Code-Mixed_Reduct_Dict.json')

In [40]:
test_reduc_dict = TranslitDict.load(src_path='/content/test_reduc_dict.dat', delimiter='\t')

In [41]:
print(f"Current Reduction Dict Length: {len(current_reduc_dict)}\nAdd Reduction Dict Length: {len(test_reduc_dict)}")

Current Reduction Dict Length: 1798
Add Reduction Dict Length: 174


In [42]:
current_reduc_dict.update(test_reduc_dict)

In [43]:
print(f"New Reduction Dict Length: {len(current_reduc_dict)} ")

New Reduction Dict Length: 1798 


In [36]:
current_reduc_dict.export('/content/Nep_Eng_Code-Mixed_Reduct_Dict.json')

In [37]:
!cp  /content/Nep_Eng_Code-Mixed_Reduct_Dict.json '/content/drive/MyDrive/MSICE/English_Nepali_CS_Data_Manual/Transliteration Dictionary/'

##CREATE REDUCED TRANSCRIPT FILE

1. Inputs are transcript file, reduction dictionary
2. Output is reduced transcript file

In [ ]:
# !cp /content/transliteration/reducers.py /content/

In [ ]:
!cp /content/drive/MyDrive/MSICE/English_Nepali_CS_Data_Manual/Transliteration\ Dictionary/Nep_Eng_Code-Mixed_Reduct_Dict.json /content/

In [ ]:
reduc_dict = TranslitDict.load(src_path = '/content/Nep_Eng_Code-Mixed_Reduct_Dict.json')
def reduce(word: str) -> str:
    reduced_word = reduc_dict.get(word)
    if reduced_word:
        return reduced_word
    return word

In [ ]:
transform_transcript(input_path = "/content/native_augmented_numless_punctless_transcript.txt", output_path = "/content/reduced_transcript.txt", transform_fn = reduce)

In [ ]:
!head -10 reduced_transcript.txt

SS_Dr_Sanjay_Sharma_2_audio_1	एकदम धैरै हुने भाको भएरपनि अम स्टटिस्टिकलि हुन्छ अम त्यो पनि हुन्छ
SS_Dr_Sanjay_Sharma_2_ps_audio_1	एकदम धैरै हुने भाको भएरपनि अम स्टटिस्टिकलि हुन्छ अम त्यो पनि हुन्छ
SS_Dr_Sanjay_Sharma_2_n_audio_1	एकदम धैरै हुने भाको भएरपनि अम स्टटिस्टिकलि हुन्छ अम त्यो पनि हुन्छ
SS_Dr_Sanjay_Sharma_2_wn_audio_1	एकदम धैरै हुने भाको भएरपनि अम स्टटिस्टिकलि हुन्छ अम त्यो पनि हुन्छ
SS_Dr_Sanjay_Sharma_2_rr_audio_1	एकदम धैरै हुने भाको भएरपनि अम स्टटिस्टिकलि हुन्छ अम त्यो पनि हुन्छ
SS_Dr_Sanjay_Sharma_2_sp_audio_1	एकदम धैरै हुने भाको भएरपनि अम स्टटिस्टिकलि हुन्छ अम त्यो पनि हुन्छ
SS_Dr_Sanjay_Sharma_2_lpf_audio_1	एकदम धैरै हुने भाको भएरपनि अम स्टटिस्टिकलि हुन्छ अम त्यो पनि हुन्छ
SS_Dr_Sanjay_Sharma_2_audio_2	नेपालमा पुल्सर् बाइक नै चोरि हुन्छ भनेर भन्ने एउटा नारटिव्ह बिकोज त्यहां मार्कट भरि पुल्सर् बाइक नै थियो त्यस्तै हो सोउ अम
SS_Dr_Sanjay_Sharma_2_ps_audio_2	नेपालमा पुल्सर् बाइक नै चोरि हुन्छ भनेर भन्ने एउटा नारटिव्ह बिकोज त्यहां मार्कट भरि पुल्सर् बाइक नै थियो त्यस्तै हो स